# LlamaParse + Cortex Search

This notebook walks through how to parse a complex report with LlamaParse, how to load it in Snowflake, and how to create a RAG via Cortex Search on top of the data loaded into Snowflake.

## Set key and import libraries
For this example, you will need a LlamaCloud API key and a Snowflake account. Get LlamaCloud API key following these [instructions](https://docs.cloud.llamaindex.ai/api_key), and signup for Snowflake [here](https://signup.snowflake.com/).

To create a new database and warehouse for storing Snowflake tables, you can run the following SQL statements in a Snowflake worksheet.

```sql
CREATE DATABASE IF NOT EXISTS SEC_10KS;

CREATE OR REPLACE WAREHOUSE llamaparse_cortex_search_wh WITH
     WAREHOUSE_SIZE='X-SMALL'
     AUTO_SUSPEND = 120
     AUTO_RESUME = TRUE
     INITIALLY_SUSPENDED=TRUE;
```

In [32]:
import os
import nest_asyncio
nest_asyncio.apply()

# llama-cloud
os.environ["LLAMA_CLOUD_API_KEY"] = "..."

# snowflake
os.environ["SNOWFLAKE_ACCOUNT"] = "..." # note: "_" can cause problems with the connection, use "-" instead
os.environ["SNOWFLAKE_USER"] = "..."
os.environ["SNOWFLAKE_PASSWORD"] = "..."
os.environ["SNOWFLAKE_ROLE"] = "..."
os.environ["SNOWFLAKE_WAREHOUSE"] = "..."
os.environ["SNOWFLAKE_DATABASE"] = "SEC_10KS" # note: make sure to use a database that already exists
os.environ["SNOWFLAKE_SCHEMA"] = "PUBLIC"

## Load Data

Use Snowflake's latest 10K or your favorite PDF.

To use Snowflake's latest 10K, download it from [here](https://d18rn0p25nwr6d.cloudfront.net/CIK-0001640147/663fb935-b123-4bbb-8827-905bcbb8953c.pdf) and rename it as `snowflake_2025_10k.pdf`.

## Parse PDF

In [2]:
from llama_cloud_services import LlamaParse

parser = LlamaParse(
    num_workers=4,       # if multiple files passed, split in `num_workers` API calls
    verbose=True,
    language="en",       # optionally define a language, default=en
)

# sync
result = parser.parse("./snowflake_2025_10k.pdf")

Started parsing the file under job_id fba19fc4-9e65-4f30-b98e-d6c0c4979b2e


In [3]:
# get the llama-index markdown documents
markdown_documents = result.get_markdown_documents(split_by_page=False)

## Convert LlamaIndex Documents to Dataframe

In [ ]:
import pandas as pd

# fields that matter only to vector/RAG helpers – we don’t need them here
_INTERNAL_KEYS_TO_SKIP = {
    "excluded_embed_metadata_keys",
    "excluded_llm_metadata_keys",
    "relationships",
    "metadata_template",
    "metadata_separator",
    "text_template",
    "class_name",
}

def documents_to_dataframe(documents):
    """Convert a list of LlamaIndex documents to a tidy pandas DataFrame,
    omitting vector-store helper fields that aren’t needed for analytics/LLM-matching.
    """
    rows = []

    for doc in documents:
        d = doc.model_dump(exclude_none=True)

        for k in _INTERNAL_KEYS_TO_SKIP:
            d.pop(k, None)

        # Pull out & flatten metadata
        meta = d.pop("metadata", {})
        d.update(meta)

        # Extract raw text
        t_res = d.pop("text_resource", None)
        if t_res is not None:
            d["text"] = t_res.get("text") if isinstance(t_res, dict) else getattr(t_res, "text", None)

        rows.append(d)

    return pd.DataFrame(rows)

In [5]:
documents_df = documents_to_dataframe(markdown_documents)

## Write DataFrame to Snowflake table

In [8]:
from snowflake.snowpark import Session

# Create Snowpark session
connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),            
    "role": os.getenv("SNOWFLAKE_ROLE"),
    "warehouse": os.getenv("SNOWFLAKE_WARHEOUSE"),
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
}

session = Session.builder.configs(connection_parameters).create()

In [9]:
# convert to Snowpark DataFrame
snowpark_df = session.create_dataframe(documents_df)

In [ ]:
# Write Snowpark DataFrame to a Snowflake table
# Use 'overwrite' to replace table or 'append' to add to existing table
snowpark_df.write.mode("overwrite").save_as_table("snowflake_10k")

## Split the text

In [ ]:
split_text_sql = """
CREATE OR REPLACE TABLE SNOWFLAKE_10K_MARKDOWN_CHUNKS AS
SELECT
    "id_" as ID,
    "file_name" as FILE_NAME,
    c.value::string as TEXT
FROM
    SNOWFLAKE_10K,
    LATERAL FLATTEN(input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(
        "text",
        'markdown',
        512,
        128
    )) c;
"""
session.sql(split_text_sql).collect()

[Row(status='Table SNOWFLAKE_10K_MARKDOWN_CHUNKS successfully created.')]

In [31]:
session.sql("SELECT * FROM SNOWFLAKE_10K_MARKDOWN_CHUNKS LIMIT 3").show()

------------------------------------------------------------------------------------------------------------------------
|"ID"                                  |"FILE_NAME"               |"TEXT"                                              |
------------------------------------------------------------------------------------------------------------------------
|9ddd8c6c-f0eb-47c0-8f5d-506d418ae28a  |./snowflake_2025_10k.pdf  |# Table of Contents                                 |
|                                      |                          |                                                    |
|                                      |                          |# UNITED STATES                                     |
|                                      |                          |                                                    |
|                                      |                          |# SECURITIES AND EXCHANGE COMMISSION                |
|                               

## Create Cortex Search Service

In [17]:
create_search_service_sql = """
CREATE OR REPLACE CORTEX SEARCH SERVICE SNOWFLAKE_10K_SEARCH_SERVICE
  ON TEXT
  ATTRIBUTES ID, FILE_NAME
  WAREHOUSE = S
  TARGET_LAG = '1 hour'
AS (
  SELECT
    ID,
    FILE_NAME,
    TEXT
  FROM SEC_10KS.PUBLIC.SNOWFLAKE_10K_MARKDOWN_CHUNKS
);
"""
session.sql(create_search_service_sql).collect()

[Row(status='Cortex search service SNOWFLAKE_10K_SEARCH_SERVICE successfully created.')]

In [18]:
from snowflake.core import Root
from typing import List
from snowflake.snowpark.session import Session

class CortexSearchRetriever:

    def __init__(self, snowpark_session: Session, limit_to_retrieve: int = 4):
        self._snowpark_session = snowpark_session
        self._limit_to_retrieve = limit_to_retrieve

    def retrieve(self, query: str) -> List[str]:
        root = Root(session)

        search_service = (root
          .databases["SEC_10KS"]
          .schemas["PUBLIC"]
          .cortex_search_services["SNOWFLAKE_10K_SEARCH_SERVICE"]
        )
        resp = search_service.search(
          query=query,
          columns=["text"],
          limit=self._limit_to_retrieve
        )

        if resp.results:
            return [curr["text"] for curr in resp.results]
        else:
            return []
        
retriever = CortexSearchRetriever(snowpark_session=session, limit_to_retrieve=5)

In [19]:
retrieved_context = retriever.retrieve("What was the total revenue (in billions) for Snowflake in FY 2024? How much of that was product revenue?")

retrieved_context

['________________\n\n(1) For the fiscal years ended January 31, 2025, 2024, and 2023, respectively, approximately 65%, 67%, and 71% of cost of product revenue represented third-party cloud infrastructure expenses incurred in connection with the customers’ use of the Snowflake platform and the deployment and maintenance of the platform on public clouds, including different regional deployments.',
 'Our revenue was $3.6 billion, $2.8 billion, and $2.1 billion for the fiscal years ended January 31, 2025, 2024, and 2023, respectively. As a result of our historical rapid growth, limited operating history, large number of new product features, including those incorporating artificial intelligence and machine learning technology (AI Technology), and unstable macroeconomic conditions, our ability to accurately forecast our future results of operations, including revenue, gross margin, remaining performance',
 'reflecting these adjustments. Our platform has been adopted by many of the world’s 

## Create a RAG

In [20]:
from snowflake.cortex import complete

class RAG:

    def __init__(self, session):
        self.session = session
        self.retriever = CortexSearchRetriever(snowpark_session=self.session, limit_to_retrieve=10)

    def retrieve_context(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        return self.retriever.retrieve(query)

    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        prompt = f"""
          You are an expert assistant extracting information from context provided.
          Answer the question in concisely, yet completely. Only use the information provided.
          Context: {context_str}
          Question:
          {query}
          Answer:
        """
        response = ""
        response = complete("claude-4-sonnet", prompt, session = session)
        return response

    def query(self, query: str) -> str:
        context_str = self.retrieve_context(query)
        return self.generate_completion(query, context_str)


rag = RAG(session)

In [21]:
response = rag.query("What was the total revenue (in billions) for Snowflake in FY 2024? How much of that was product revenue?")

In [22]:
from IPython.display import Markdown

display(Markdown(response))

Based on the provided context, for fiscal year 2024 (ended January 31, 2024):

**Total Revenue:** $2.8 billion

**Product Revenue:** $2,666,849 thousand, which equals approximately $2.67 billion

Product revenue represented 95% of total revenue, while professional services and other revenue made up the remaining 5% ($139,640 thousand or approximately $0.14 billion).